In [ ]:
from biolearn.data_library import DataLibrary
import pandas as pd
# https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE19711

data = DataLibrary().get("GSE19711").load()
data.metadata, data.dnam

#elastic for selection, boost for prediction

In [80]:
import numpy as np
X = data.dnam.transpose()
X_df = pd.DataFrame(X)
y = data.metadata['age']
y = pd.DataFrame(y)
cb= pd.concat([X_df, y], axis=1)
cb.shape, X_df, y

((540, 27579),
 id         cg00000292  cg00002426  cg00003994  cg00005847  cg00006414  \
 GSM491937    0.756367    0.797859    0.068605    0.131004    0.076355   
 GSM491938    0.834702    0.859538    0.067469    0.197486    0.096817   
 GSM491939    0.774165    0.769661    0.056937    0.140949    0.156980   
 GSM491940    0.799517    0.854328    0.063802    0.168209    0.086761   
 GSM491941    0.819867    0.853270    0.061275    0.137825    0.079826   
 ...               ...         ...         ...         ...         ...   
 GSM492472    0.850640    0.811812    0.046001    0.165387    0.074648   
 GSM492473    0.785949    0.863659    0.065620    0.149490    0.078410   
 GSM492474    0.827493    0.815332    0.055990    0.155754    0.091335   
 GSM492475    0.817919    0.860943    0.062512    0.129134    0.065681   
 GSM492476    0.555077    0.511268    0.210708    0.066549    0.056630   
 
 id         cg00007981  cg00008493  cg00008713  cg00009407  cg00010193  ...  \
 GSM491937    0.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNetCV

# Assuming X_genomic is your genomic dataset and y_age is the corresponding age labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.2, random_state=42)

# Initialize pipeline with imputer and scaler
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Use mean imputation for missing values
    ('scaler', StandardScaler())
])

# Fit and transform training data
X_train_scaled = pipeline.fit_transform(X_train)

# Transform test data
X_test_scaled = pipeline.transform(X_test)

# Initialize and fit the ElasticNetCV model
alpha_values = np.logspace(-3, 3, 10)  # Range of alpha values to search
l1_ratio_values = [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]  # Range of l1_ratio values to search
elasticnet_cv = ElasticNetCV(alphas=alpha_values, l1_ratio=l1_ratio_values, cv=5)
elasticnet_cv.fit(X_train_scaled, y_train)

# Print the selected alpha and l1_ratio values
print("Selected alpha:", elasticnet_cv.alpha_)
print("Selected l1_ratio:", elasticnet_cv.l1_ratio_)

# Evaluate the model on the test set
test_score = elasticnet_cv.score(X_test_scaled, y_test)
print("Test Set R^2 Score:", test_score)


In [78]:
#####  feature selection

from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
X_df, y = make_regression(n_samples=100, n_features=100, noise=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.1, random_state=42)
model = ElasticNet(alpha=0.4, l1_ratio=0.6) 
model.fit(X_train, y_train)
coefficients = model.coef_

# Print the number of non-zero coefficients
num_nonzero = np.count_nonzero(coefficients)
print("Number of non-zero coefficients:", num_nonzero)

non_zero_columns = pd.DataFrame(X_train).columns[coefficients != 0]
print(non_zero_columns)
X_train_df = pd.DataFrame(X_train, columns=[f"Feature_{i}" for i in range(X_train.shape[1])])

non_zero_columns = X_train_df.columns[model.coef_ != 0]
print("Column names of non-zero coefficients:", non_zero_columns)


Number of non-zero coefficients: 93
Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 12, 13, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
       38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56,
       57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 71, 72, 73, 75, 76,
       77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 88, 90, 91, 92, 93, 94, 95, 96,
       97, 98, 99],
      dtype='int64')
Column names of non-zero coefficients: Index(['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3', 'Feature_4',
       'Feature_5', 'Feature_6', 'Feature_7', 'Feature_8', 'Feature_9',
       'Feature_11', 'Feature_12', 'Feature_13', 'Feature_15', 'Feature_16',
       'Feature_17', 'Feature_18', 'Feature_19', 'Feature_20', 'Feature_21',
       'Feature_22', 'Feature_23', 'Feature_24', 'Feature_25', 'Feature_26',
       'Feature_27', 'Feature_28', 'Feature_29', 'Feature_30', 'Feature_31',
       'Feature_32', 'Feature_33', 'F

In [86]:
non_zero_coefficients = coefficients[non_zero_indices]
non_zero_feature_names = X_df.columns[non_zero_indices]
for coef, feature_name in zip(non_zero_coefficients, non_zero_feature_names):
    print(f"{feature_name}: {coef:.4f}")

cg00000292: 0.6474
cg00003994: 4.7556
cg00005847: 5.8235
cg00006414: 30.5445
cg00007981: 28.6039
cg00008493: -0.3353
cg00008713: 7.7542
cg00009407: 2.8503
cg00011459: -0.0000
cg00012199: 1.9435
cg00012386: 2.2607
cg00012792: 1.8734
cg00013618: 0.0000
cg00014085: -3.9383
cg00014837: 1.5995
cg00015770: 1.7930
cg00016968: 4.2828
cg00019495: -3.4246
cg00020533: -1.5448
cg00021527: 3.0546
cg00022606: 3.1943
cg00022866: 2.3855
cg00024396: 0.1302
cg00025138: 4.1996
cg00025991: -5.0169
cg00027083: -5.3641
cg00027674: 4.4294
cg00029931: -3.1816
cg00030047: -0.1563
cg00031162: -0.0948
cg00032227: -0.8227
cg00032666: -0.1911
cg00033773: 1.2034
cg00035347: -1.8498
cg00035623: 3.3269
cg00037763: 3.3437
cg00037940: -7.2966
cg00040861: -3.4526
cg00040873: 3.1898
cg00041575: -1.9396
cg00042156: -1.6376
cg00043004: -2.0238
cg00043080: -3.7378
cg00044245: -0.0000
cg00044729: 55.8326
cg00047050: 2.4033
cg00049986: -0.1999
cg00050312: 41.4529
cg00051623: -2.9348
cg00053292: -2.4973
cg00053647: 2.0630
cg00

In [89]:
X_choose = X_df[["cg00000292",	"cg00003994",	"cg00005847",	"cg00006414",	"cg00007981",	"cg00008493",	"cg00008713",	"cg00009407",	"cg00011459",	"cg00012199",	"cg00012386",	"cg00012792",	"cg00013618",	"cg00014085",	"cg00014837",	"cg00015770",	"cg00016968",	"cg00019495",	"cg00020533",	"cg00021527",	"cg00022606",	"cg00022866",	"cg00024396",	"cg00025138",	"cg00025991",	"cg00027083",	"cg00027674",	"cg00029931",	"cg00030047",	"cg00031162",	"cg00032227",	"cg00032666",	"cg00033773",	"cg00035347",	"cg00035623",	"cg00037763",	"cg00037940",	"cg00040861",	"cg00040873",	"cg00041575",	"cg00042156",	"cg00043004",	"cg00043080",	"cg00044245",	"cg00044729",	"cg00047050",	"cg00049986",	"cg00050312",	"cg00051623",	"cg00053292",	"cg00053647",	"cg00054706",	"cg00055233",	"cg00056767",	"cg00057593",	"cg00058938",	"cg00059225",	"cg00059424",	"cg00059740",	"cg00059930",	"cg00060762",	"cg00060882",	"cg00061059",	"cg00061629",	"cg00063144",	"cg00065385",	"cg00065408",	"cg00066153",	"cg00066816",	"cg00067471",	"cg00069261",	"cg00071998",	"cg00072216",	"cg00075967",	"cg00076645",	"cg00077457",	"cg00078194",	"cg00078867",	"cg00079056",	"cg00080012",	"cg00081935",	"cg00081975",	"cg00083720",	"cg00083937",	"cg00084687",	"cg00089071",	"cg00090147"]]

In [135]:
###### XGBoost

import xgboost as xgb

#X_choose, y = make_regression(n_samples=100, n_features=100, noise=0.1, random_state=42)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_choose, y, test_size=0.2, random_state=42)

# Train an XGBoost model
xgb_model = xgb.XGBRegressor(n_estimators=10000, learning_rate=0.15, max_depth=2, reg_alpha=0.7, reg_lambda=0.1)
xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='rmse', early_stopping_rounds=10, verbose=False)

# Evaluate the model
train_score = xgb_model.score(X_train, y_train)
test_score = xgb_model.score(X_test, y_test)
print("XGBoost model train score:", train_score)
print("XGBoost model test score:", test_score)

XGBoost model train score: 0.9901515471112556
XGBoost model test score: 0.8338680125606357


/Users/petra/Desktop/study/UCSF/datasci 223 python/datasci_223/.conda/lib/python3.11/site-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/Users/petra/Desktop/study/UCSF/datasci 223 python/datasci_223/.conda/lib/python3.11/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


In [122]:
from sklearn.metrics import mean_squared_error

# Predictions on the training and testing sets
y_train_pred = xgb_model.predict(X_train)
y_test_pred = xgb_model.predict(X_test)

# Calculate the Mean Squared Error
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

print("MSE on training set:", mse_train)
print("MSE on testing set:", mse_test)

from sklearn.metrics import r2_score, roc_auc_score

# Calculate R-squared (R2) score
r2 = r2_score(y_test, y_test_pred)
print("R-squared (R2) score:", r2)


MSE on training set: 197.88939012726706
MSE on testing set: 2611.7418075311098


In [142]:
#########random forest

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 150],  # Number of trees in the forest
    'max_depth': [None, 10, 20],      # Maximum depth of the trees
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]     # Minimum number of samples required to be at a leaf node
}

# Instantiate the Random Forest regressor
rf_model = RandomForestRegressor(random_state=42)

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params = grid_search.best_params_
best_rf_model = grid_search.best_estimator_

print("Best parameters:", best_params)

# Evaluate the best model
y_train_pred = best_rf_model.predict(X_train)
y_test_pred = best_rf_model.predict(X_test)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

print("MSE on training set with best model:", mse_train)
print("MSE on testing set with best model:", mse_test)


Best parameters: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 50}
MSE on training set with best model: 2760.316786133939
MSE on testing set with best model: 7296.453596243734


In [167]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Instantiate a new Random Forest model with the best parameters
best_rf_model = RandomForestRegressor(n_estimators=1000,
                                       max_depth=5,
                                       min_samples_split=2,
                                       min_samples_leaf=4,
                                       random_state=10)

# Fit the model to the training data
best_rf_model.fit(X_train, y_train)

# Evaluate the model on the testing data
y_test_pred = best_rf_model.predict(X_test)
mse_test = mean_squared_error(y_test, y_test_pred)

print("MSE on testing set with best model:", mse_test)


MSE on testing set with best model: 6544.6839326361205


In [170]:
####### SVM

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_absolute_error

# Instantiate the SVM regressor with the desired hyperparameters
svm_model = SVR(kernel='rbf', C=1.0, epsilon=0.1)

# Fit the SVM model to the training data
svm_model.fit(X_train, y_train)

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'kernel': ['linear', 'rbf', 'poly'],
    'C': [0.1, 1, 10],
    'epsilon': [0.1, 0.2, 0.5]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params = grid_search.best_params_
best_svm_model = grid_search.best_estimator_

print("Best parameters:", best_params)

# Evaluate the best model
y_train_pred = best_svm_model.predict(X_train)
y_test_pred = best_svm_model.predict(X_test)

train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

print("R-squared on training set with best model:", train_r2)
print("R-squared on testing set with best model:", test_r2)
print("MAE on training set with best model:", train_mae)
print("MAE on testing set with best model:", test_mae)
print("MSE on training set with best model:", train_mse)
print("MSE on testing set with best model:", test_mse)


Best parameters: {'C': 10, 'epsilon': 0.1, 'kernel': 'linear'}
R-squared on training set with best model: 0.9999995015487311
R-squared on testing set with best model: 0.7136437588710891
MAE on training set with best model: 0.10007764714881104
MAE on testing set with best model: 59.23761991852349
MSE on training set with best model: 0.010015605367927432
MSE on testing set with best model: 4501.77342925488
